In [ ]:
!pip install beautifulsoup4
!pip install requests

In [ ]:
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Fungsi untuk mengekstrak data dari setiap halaman ANTARA News dengan retry mechanism
def extract_data_from_antaranews(page, retries=3):
    url = f"https://www.antaranews.com/search?q=Badan+Penyelenggara+Jaminan+Sosial+%28BPJS%29&page={page}"

    # Mengulang hingga jumlah retry habis
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)

            if response.status_code != 200:
                print(f"Failed to retrieve page {page}")
                return None

            soup = BeautifulSoup(response.text, "html.parser")
            articles = soup.find_all("div", {"class": "card__post card__post-list card__post__transition mt-30"})

            data = []
            for article in articles:
                # Judul dan link
                title_tag = article.find("div", {"class": "card__post__title"})
                title = title_tag.get_text().strip() if title_tag else "No title"
                link = title_tag.find("a")['href'] if title_tag and title_tag.find("a") else "No link"

                # Tanggal
                date_tag = article.find("span", {"class": "text-dark text-capitalize"})
                date = date_tag.get_text().strip() if date_tag else "No date"

                # Filter hanya ambil judul yang mengandung kata "BPJS"
                if "bpjs" in title.lower():
                    data.append({
                        'Judul': title,
                        'Link': link,
                        'Tanggal': date
                    })

            return pd.DataFrame(data)

        except requests.ConnectionError as e:
            print(f"Connection error on page {page}, attempt {attempt + 1} of {retries}")
            time.sleep(2)  # Tunggu sebelum mencoba lagi

    print(f"Failed to retrieve page {page} after {retries} attempts.")
    return None

# Fungsi untuk melakukan scraping semua halaman hingga tidak ada data lagi
def scrape_all_pages():
    page = 1
    all_data = []

    while True:
        print(f"Scraping page {page}...")
        page_data = extract_data_from_antaranews(page)

        # Jika tidak ada artikel di halaman tersebut, berhenti
        if page_data is None or page_data.empty:
            print(f"No more data found on page {page}. Stopping.")
            break

        all_data.append(page_data)
        page += 1

    # Menggabungkan semua data dari setiap halaman
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

# Scraping semua halaman ANTARA News
print("Starting to scrape ANTARA News...")
antaranews_data = scrape_all_pages()

# Output hasil
if not antaranews_data.empty:
    for index, item in antaranews_data.iterrows():
        print(f"Judul: {item['Judul']}")
        print(f"Link: {item['Link']}")
        print(f"Tanggal: {item['Tanggal']}")
        print("-" * 50)
else:
    print("No data scraped.")


Starting to scrape ANTARA News...
Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 4

In [ ]:
import pandas as pd

# Step 1: Mengonversi hasil scraping dari ANTARA News ke dalam DataFrame
# Pastikan variabel yang digunakan benar, yaitu 'antaranews_data' bukan 'liputan6_data'
df = pd.DataFrame(antaranews_data)  # Menggunakan data yang sudah di-scrape dari ANTARA

# Step 2: Mengekspor DataFrame ke CSV dengan pemisah ;
df.to_csv('Antara_bpjs_articles.csv', sep=';', index=False)

print("Data berhasil diekspor ke 'Antara_bpjs_articles.csv' dengan pemisah ;")


Data berhasil diekspor ke 'Antara_bpjs_articles.csv' dengan pemisah ;
